<h1>Casestudy Eric Bühler - Aachen </h1>

In [4]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import collections
from datetime import datetime, timedelta
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
import numpy as np
import pandas as pd
import configparser
import openai


In [5]:
def open_AI_call():
    # Your OpenAI API key
        api_key = 'key'

        # Initialize the OpenAI client
        openai.api_key = api_key

        # Define the messages to send to the API based on your previous message
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Welche PLZ hat dieser Stadtteil von Aachen? f{stadtteil} Bitte schreib nur die Zahl"}
        ]

        # Call the OpenAI API with the chat completion method
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",  # Specify the model to use, you can change this as needed
            messages=messages,
            max_tokens=10,  # Adjust the number of tokens as needed
            n=1,
            stop=None,
            temperature=0.7
        )

        # Extract the text response
        plz = response.choices[0].message['content'].str

In [6]:


def scrape_wgs(pages=2):
    anzeigen=[]
    for i in range(pages):
        url = f"https://www.wg-gesucht.de/wg-zimmer-in-Aachen.1.0.1.{i}.html?offer_filter=1&city_id=1&sort_order=0&noDeact=1&categories%5B%5D=0&pagination=1&pu="
        # Sends a GET request to the URL to retrieve its content
        response = requests.get(url)
        assert response.status_code == 200
        #print(response.text)
        # Parse results
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting all Anzeigen
        anzeigen_raw = soup.find_all('div', class_='col-sm-8 card_body')
        # Loop through each anzeige section and build dictionaries of them
        raw_text = ''
        for anzeige in anzeigen_raw:
            titel = anzeige.find('a').get_text().replace('\n', '').strip()
            preis = anzeige.find('div', class_='col-xs-3').get_text().replace('€','').strip()
            frei_ab = anzeige.find('div', class_='col-xs-5 text-center').get_text().replace('\n', '').split('-')[0].strip()
            qm = anzeige.find('div', class_='col-xs-3 text-right').get_text().replace('\n', '').replace('m²','').strip()
            wg_groesse = anzeige.find('div', class_='col-xs-11').get_text().split('er')[0].split()[0]
            stadtteil = anzeige.find('div', class_='col-xs-11').get_text().strip().split('|')[1].replace('\n', '').strip().split()
            #uncomment if i get rich
            #plz = open_AI_call()
            
            online_seit = anzeige.find('div', class_='col-xs-9').get_text().split('Online:')[1].strip()
            if "Sekunde" in online_seit or "Sekunden" in online_seit:
                online_seit="00:00"

            if "Minuten" in online_seit or "Minute" in online_seit:
                minutes = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"00:{minutes.zfill(2)}"  # Ensuring minutes are two digits

            if "Stunden" in online_seit or "Stunde" in online_seit:
                hours = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"{hours.zfill(2)}:00"  # Ensuring hours are two digits

            if "Tag" in online_seit or "Tage" in online_seit:
                days = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"{days*24}:00"

            
            anzeigen.append({'titel': titel,'preis' : preis,'frei_ab':frei_ab, 'qm': qm,'wg_groesse':wg_groesse,'stadtteil':stadtteil,'online_seit':online_seit})
    return pd.DataFrame(anzeigen)

# Display the DataFrame
display(scrape_wgs(4))

,titel,preis,frei_ab,qm,wg_groesse,stadtteil,online_seit
0,2 schöne möblierte Zimmer in Aachen-Mitte mit ...,279,09.05.2024,16,14,"[Aachen, Aachen]",00:01
1,WG-Zimmer in bester Lage für Studenten,200,01.07.2024,14,6,"[Aachen, Aachen]",00:02
2,Schönes Zimmer in Studentenwohnheim zu vermieten,120,12.06.2024,11,11,"[Aachen, Aachen]",00:02
3,Möbliertes Zimmer in Uninähe,265,12.06.2024,21,11,"[Aachen, Aachen]",00:02
4,Studentenwohnheim mit freundschaftlicher Gemei...,215,12.06.2024,20,11,"[Aachen, Aachen]",00:03
5,1 Zimmer in luxeriöser 3er WG ab Juli frei: Ei...,500,01.07.2024,17,3,"[Aachen, Aachen]",00:05
6,Schönes Zimmer in 3er WG am Westpark,404,11.06.2024,12,3,"[Aachen, West]",00:05
7,"Fachwerkhaus in Roetgen.Wohnen,da wo andere Ur...",300,12.06.2024,14,2,"[Aachen, Roetgen]",00:07
8,Neugebaute Studentenzimmer in bester Lage,250,12.06.2024,14,17,"[Aachen, Aachen]",00:08
9,Zimmer in bester Unilage ab dem 1.9. zu vermie...,235,01.09.2024,15,10,"[Aachen, Aachen]",00:09
